In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

! wget https://raw.githubusercontent.com/kumgleb/NAS_Test_Task/master/model.py -O model.py

from model import SuperNet

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


--2020-08-10 10:27:11--  https://raw.githubusercontent.com/kumgleb/NAS_Test_Task/master/model.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6820 (6.7K) [text/plain]
Saving to: ‘model.py’

model.py            100%[===================>]   6.66K  --.-KB/s    in 0s      

2020-08-10 10:27:11 (49.2 MB/s) - ‘model.py’ saved [6820/6820]



# Notes:
* Plot super acc vs stand-alone acc.
* Take note about possible coodaptation in 'full' approach then evaluating on stand-alone model.
* Take note about possible path dropout rate scheduling
* Explore path rdopout rate and one-shot vs stand-alone accuracy.


# **Data**:
MNIST dataset is used for model train and evaluation. <br>
Basic MNIST test set splitted equally on validation and test sets.
* SuperNet is trained on a MNIST train set.
* SuperNet is evaluated on MNIST a validation set.
* SubNet's are evaluated on a validation set.
* Final arcitectures traind form scratch evaluated on a test set.


In [2]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

X_val, X_test, y_val, y_test = train_test_split(X_test, y_test,
                                                stratify=y_test,
                                                test_size=0.5,
                                                random_state=42)

X_train = X_train.reshape(-1, 28, 28, 1) / 255
X_val = X_val.reshape(-1, 28, 28, 1) / 255
X_test = X_test.reshape(-1, 28, 28, 1) / 255

print('')
print('Train shape: ', X_train.shape)
print('Validation shape: ', X_val.shape)
print('Test shape: ', X_test.shape )

11493376/11490434 [==============================] - 0s 0us/step

Train shape:  (60000, 28, 28, 1)
Validation shape:  (5000, 28, 28, 1)
Test shape:  (5000, 28, 28, 1)




---



# **Experiments**:

In the following sections experiments with SuperNet are provided:
1. Main part consist of experiments provided in task. SuperNet trained with subsampling on SubNet's and their weights update.
2. Additional part consist of few more experiments with different SuperNet training approach. SuperNet traindew with path dropout.

In all experiments layers parameters, such as number of filters in convolution layers and number on neurons in dense layer remains fixed. <br>
Learning rate, batch size and number of epoces remains the same as well.

In [3]:
# Fixed parameters
N_FILTERS = 16 # number of filters in all convolution layers
N_DENSE = 64 # number of neurons in dense layer
N_ClASSES = 10 # number of classes in dataset

N_EPOCHS = 3
BATCH_SIZE = 256
LEARNING_RATE = 1e-3

# 1. Main part:
In a part below followng experiments are provided:
1. Training of a SuperNet with `random_subnet` strategy (during training different subnets parameters are updated, subnets selected randomly after defined number of batches, which regulated by `N_BTHCH_TO_SWITCH` parameter).
2. Then SuperNet is trained, SubNet's are sampled form it and evaluated on a validation set. Top-1 accuracy on a validation set are difined.
3. Arcitectures form above are retrained with random initialization, then accuracys evaluated on a test set. Top-1 accuracy on a test set are difined.


In [5]:
model = SuperNet(N_FILTERS, N_DENSE, N_ClASSES)

1. Training of a SuperNet with `random_subnet` strategy:

In [7]:
N_BTHCH_TO_SWITCH = 128 # number of batches before model randomly swith to another SubNet during training

model.fit(X_train, y_train, N_EPOCHS, BATCH_SIZE, N_BTHCH_TO_SWITCH, LEARNING_RATE,
          validation_data=(X_val, y_val), train_mode='random_subnet', verbosity=1) # correct after model.py update

Epoch: 0, SuperNet_train_loss: 0.2070, SuperNet_train_accuracy: 94.38%
Epoch: 0
  SuperNet validation accuracy: 97.380%
Epoch: 1, SuperNet_train_loss: 0.2588, SuperNet_train_accuracy: 96.26%
Epoch: 1
  SuperNet validation accuracy: 97.220%
Epoch: 2, SuperNet_train_loss: 0.0801, SuperNet_train_accuracy: 97.44%
Epoch: 2
  SuperNet validation accuracy: 97.560%


2. Sampling and evaluating of SubNet's on a validation set:

In [8]:
def evaluate_sample(sample_model, X_test, y_test):
  logits = sample_model(X_test)
  prediction = tf.argmax(logits, axis=1)
  accuracy = tf.keras.metrics.Accuracy()(prediction, y_test)
  return accuracy

In [9]:
for idx in [1, 2, 3, 4]:
  sample = model._sample_subnet(idx)
  test_accuracy = evaluate_sample(sample, X_val, y_val)
  print(f'SubNet: {idx}, test accuracy: {test_accuracy:.2%}')

SubNet: 1, test accuracy: 97.00%
SubNet: 2, test accuracy: 94.72%
SubNet: 3, test accuracy: 98.26%
SubNet: 4, test accuracy: 92.82%


As one can see SubNet with arcitecture 3 can be ranked as top-1  arcitecture with 98.26% accuracy on a validation set. <br>
It also overperforms full SuperNet. <br>
Corresponding arcitecture provides bellow:

In [10]:
sample = model._sample_subnet(3)
sample.summary()

Model: "subnet_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1564 (InputLayer)         [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
5x5_1_conv_subnets_3_and_4 (Con (None, 28, 28, 16)   416         input_1564[0][0]                 
__________________________________________________________________________________________________
tf_op_layer_ZerosLike_1204 (Ten [(None, 28, 28, 16)] 0           5x5_1_conv_subnets_3_and_4[1305][
__________________________________________________________________________________________________
concatenate_3126 (Concatenate)  (None, 28, 28, 32)   0           tf_op_layer_ZerosLike_1204[0][0] 
                                                                 5x5_1_conv_subnets_3_and_4

3. Training of SubNet's form scratch:

In [47]:
def train_and_evaluate_from_scratch(X_train, y_train,
                                    X_val, y_val,
                                    X_test, y_test,
                                    n_filters, n_dense, n_classes, lr, batch_size,
                                    patience=2, n_loops_avg=5):
  for model_idx in [1, 2, 3, 4]:
    accuracy = []
    for k in range(n_loops_avg):
      model = SuperNet(n_filters, n_dense, n_classes)
      subnet = model._sample_subnet(model_idx)
      subnet.compile(optimizer=tf.keras.optimizers.Adam(lr),
                    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                    metrics=['accuracy'])
      subnet.fit(X_train, y_train,
                epochs=50,
                batch_size=batch_size,
                validation_data=(X_val, y_val),
                verbose=0,
                callbacks=tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                          patience=patience,
                                                          verbose=0,
                                                          restore_best_weights=True))
      eval_acc = subnet.evaluate(X_test, y_test)
      accuracy.append(eval_acc[1])

    accuracy = np.array(accuracy)
    print(f'SubNet {model_idx} test accuracy: {accuracy.mean():.2%}+-{accuracy.std():.2%}')

In [ ]:
train_and_evaluate_from_scratch(X_train, y_train,
                                X_val, y_val,
                                X_test, y_test,
                                N_FILTERS, N_DENSE, N_ClASSES, LEARNING_RATE, BATCH_SIZE)

157/157 [==============================] - 0s 2ms/step - loss: 0.0326 - accuracy: 0.9896
SubNet 1 test accuracy: 98.71%+-0.19%
157/157 [==============================] - 0s 2ms/step - loss: 0.0385 - accuracy: 0.9862
SubNet 2 test accuracy: 98.78%+-0.10%
157/157 [==============================] - 0s 2ms/step - loss: 0.0304 - accuracy: 0.9904
SubNet 3 test accuracy: 98.85%+-0.12%
157/157 [==============================] - 61s 389ms/step - loss: 0.0272 - accuracy: 0.9888


# 2. Additional part:

In [9]:
model = SuperNet(N_FILTERS, N_DENSE, N_ClASSES)

In [10]:
PATH_DROPOUT_P = 0.2

model.fit(X_train, y_train, N_EPOCHS, BATCH_SIZE, N_BTHCH_TO_SWITCH, LEARNING_RATE,
          validation_data=(X_val, y_val), path_dropout_prob=PATH_DROPOUT_P, train_mode='path_dropout', verbosity=1) # correct after model.py update

TypeError: ignored